In [50]:
import VCF
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.max_rows', 999)
pd.options.display.max_columns = None

In [35]:
hg38_gtf = pd.read_csv('/Users/lincoln.harris/code/cerebra/cerebra/wrkdir/hg38-plus.gtf', delimiter = '\t', header = None)
hg38_gtf

,0,1,2,3,4,5,6,7,8
0,chr1,unknown,exon,11874,12227,.,+,.,"gene_id ""DDX11L1""; gene_name ""DDX11L1""; transc..."
1,chr1,unknown,exon,12613,12721,.,+,.,"gene_id ""DDX11L1""; gene_name ""DDX11L1""; transc..."
2,chr1,unknown,exon,13221,14409,.,+,.,"gene_id ""DDX11L1""; gene_name ""DDX11L1""; transc..."
3,chr1,unknown,exon,14362,14829,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
4,chr1,unknown,exon,14970,15038,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
5,chr1,unknown,exon,15796,15947,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
6,chr1,unknown,exon,16607,16765,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
7,chr1,unknown,exon,16858,17055,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
8,chr1,unknown,exon,17233,17368,.,-,.,"gene_id ""WASH7P""; gene_name ""WASH7P""; transcri..."
9,chr1,unknown,exon,17369,17436,.,-,.,"gene_id ""MIR6859-3""; gene_name ""MIR6859-3""; tr..."


In [36]:
returnStr = str(hg38_gtf.iloc[0][8])     # keep just the gene name / metadata col
returnStr = returnStr.split(';')[1]
returnStr = returnStr.strip(' gene_name')
returnStr = returnStr.strip(' ')
returnStr = returnStr.strip('"')
returnStr

'DDX11L1'

In [37]:
def get_gene_name(posString):
    """ return the gene name from a given genome position string
       (ie. '1:21890111-21890111'), by querying the hg38-plus.gtf """

    chrom = posString.split(':')[0] # work on posString
    posString_remove = posString.split(':')[1]
    lPosition = posString_remove.split('-')[0] 
    rPosition = posString_remove.split('-')[1] 

    # work on hg38_gtf
    chromStr = 'chr' + str(chrom)
    hg38_gtf_filt = hg38_gtf.where(hg38_gtf[0] == chromStr).dropna()
    hg38_gtf_filt = hg38_gtf_filt.where(hg38_gtf_filt[3] <= int(lPosition)).dropna() # lPos good
    hg38_gtf_filt = hg38_gtf_filt.where(hg38_gtf_filt[4] >= int(rPosition)).dropna() # rPos good

    try:
        returnStr = str(hg38_gtf_filt.iloc[0][8])	# keep just the gene name / metadata col
        returnStr = returnStr.split(';')[1]
        returnStr = returnStr.strip(' gene_name')
        returnStr = returnStr.strip(' ')
        returnStr = returnStr.strip('"')
    except IndexError:
        returnStr = ''

    return returnStr


In [82]:
artifical_gct = pd.read_csv('/Users/lincoln.harris/code/cerebra/cerebra/wrkdir/geneCellMutationCounts_artifical_TEST.csv')
artifical_gct

,cellName,BRAF,EGFR,KRAS
0,artificalVCF/B1_1000,1,0,0
1,artificalVCF/B1_1001,1,0,0
2,artificalVCF/B1_1002,1,0,0
3,artificalVCF/B1_1003,1,0,0
4,artificalVCF/B1_1004,1,0,0
5,artificalVCF/E1_1000,0,1,0
6,artificalVCF/E1_1001,0,1,0
7,artificalVCF/E1_1002,0,1,0
8,artificalVCF/E1_1003,0,1,0
9,artificalVCF/E1_1004,0,1,0


In [55]:
# what does a 'real' geneCellMutationsCounts table look like? 
real_gct = pd.read_csv('/Users/lincoln.harris/Desktop/geneCellMutationCounts_3.27.19.csv')
real_gct

,Unnamed: 0,LY6E,HEXDC,EGFR,HLA-C,NANS,DGCR6L,ETFB,GTF2IRD1,TMEM51,...,SYT13,LRRC73,ICAM5,ELANE,CHD5,TNFAIP8L3,IRF5,EPB41L4B,ATP1A3,C15orf40
0,A10_1001000407,2,1,32,18,2,7,6,3,3,...,0,0,0,0,0,0,0,0,0,0
1,A10_1001000408,2,0,0,18,2,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,A10_1001000412,2,0,38,18,2,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A10_B000863,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A10_B001007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,A10_B001470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,A10_B001474,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A10_B001545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,A10_B001548,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,A10_B001554,0,0,0,27,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
real_gct.columnsumns

Index(['Unnamed: 0', 'LY6E', 'HEXDC', 'EGFR', 'HLA-C', 'NANS', 'DGCR6L',
       'ETFB', 'GTF2IRD1', 'TMEM51',
       ...
       'SYT13', 'LRRC73', 'ICAM5', 'ELANE', 'CHD5', 'TNFAIP8L3', 'IRF5',
       'EPB41L4B', 'ATP1A3', 'C15orf40'],
      dtype='object', length=13917)

In [57]:
# what are these Unnamed cols? 
real_gct['Unnamed: 138']

0       1542
1        849
2        633
3       1911
4       1904
5       1900
6       1902
7       1867
8       1878
9       1644
10      1750
11      1822
12      1876
13      1895
14      1907
15      1896
16      1851
17      1861
18      1847
19      1860
20      1866
21      1840
22      1908
23       561
24       896
25       586
26      1897
27      1882
28      1808
29      1902
        ... 
9769    1604
9770    1872
9771    1527
9772    1853
9773    1091
9774     693
9775    1808
9776    1871
9777    1865
9778    1884
9779    1914
9780    1878
9781    1914
9782    1888
9783    1897
9784    1862
9785    1772
9786    1786
9787    1868
9788    1860
9789    1903
9790    1880
9791    1852
9792    1914
9793    1854
9794    1901
9795    1870
9796    1906
9797    1477
9798    1870
Name: Unnamed: 138, Length: 9799, dtype: int64

In [58]:
real_gct['Unnamed: 138']

0       1542
1        849
2        633
3       1911
4       1904
5       1900
6       1902
7       1867
8       1878
9       1644
10      1750
11      1822
12      1876
13      1895
14      1907
15      1896
16      1851
17      1861
18      1847
19      1860
20      1866
21      1840
22      1908
23       561
24       896
25       586
26      1897
27      1882
28      1808
29      1902
        ... 
9769    1604
9770    1872
9771    1527
9772    1853
9773    1091
9774     693
9775    1808
9776    1871
9777    1865
9778    1884
9779    1914
9780    1878
9781    1914
9782    1888
9783    1897
9784    1862
9785    1772
9786    1786
9787    1868
9788    1860
9789    1903
9790    1880
9791    1852
9792    1914
9793    1854
9794    1901
9795    1870
9796    1906
9797    1477
9798    1870
Name: Unnamed: 138, Length: 9799, dtype: int64

In [67]:
real_gct.rename(columns={'Unnamed: 0' :'cells'}, inplace=True)
real_gct

,cells,LY6E,HEXDC,EGFR,HLA-C,NANS,DGCR6L,ETFB,GTF2IRD1,TMEM51,...,SYT13,LRRC73,ICAM5,ELANE,CHD5,TNFAIP8L3,IRF5,EPB41L4B,ATP1A3,C15orf40
0,A10_1001000407,2,1,32,18,2,7,6,3,3,...,0,0,0,0,0,0,0,0,0,0
1,A10_1001000408,2,0,0,18,2,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,A10_1001000412,2,0,38,18,2,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A10_B000863,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A10_B001007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,A10_B001470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,A10_B001474,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A10_B001545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,A10_B001548,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,A10_B001554,0,0,0,27,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
cols = real_gct.columns
dropList = []

for col in cols:
    if 'Unnamed' in col:
        dropList.append(col)

dropList

['Unnamed: 41', 'Unnamed: 138']

In [74]:
real_gct = real_gct.drop(columns=dropList)
real_gct

,cells,LY6E,HEXDC,EGFR,HLA-C,NANS,DGCR6L,ETFB,GTF2IRD1,TMEM51,...,SYT13,LRRC73,ICAM5,ELANE,CHD5,TNFAIP8L3,IRF5,EPB41L4B,ATP1A3,C15orf40
0,A10_1001000407,2,1,32,18,2,7,6,3,3,...,0,0,0,0,0,0,0,0,0,0
1,A10_1001000408,2,0,0,18,2,7,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,A10_1001000412,2,0,38,18,2,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A10_B000863,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A10_B001007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,A10_B001470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,A10_B001474,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A10_B001545,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,A10_B001548,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,A10_B001554,0,0,0,27,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# verify
cols = real_gct.columns

for col in cols:
    if 'Unnamed' in col:
        print(col)